<a href="https://colab.research.google.com/github/SergeyKamenshchikov/CRIP_recognition/blob/main/accelerator_%D0%A6%D0%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install libraries

In [1]:
!pip3 install pdfplumber
!pip3 install openai==1.55.3 -q

!pip3 install pdf2image
!apt-get install poppler-utils
!pip3 install easyocr

!apt-get update && apt-get install -y pandoc
!pip3 install python-pptx pandas tabulate
!pip3 install pypandoc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 16.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (2,094 kB/s)
Selecting previously unselected package poppler-utils.


##### Import libraries:

In [2]:
from openai import OpenAI

from PIL import Image
import os, time, json

import numpy as np
import pandas as pd

from tqdm import tqdm
from pptx import Presentation

from google.colab import drive
from google.colab import output
from google.colab import files

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

import nltk, re
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

from tqdm import tqdm

from pdf2image import convert_from_path
import easyocr, pdfplumber

import warnings, logging

import threading, queue
from collections import namedtuple

import base64
import tempfile
import pypandoc

from PIL import Image

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


##### Ignore warnings

In [3]:
warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)
Image.MAX_IMAGE_PIXELS = None

##### Keys

In [4]:
os.environ["OPENAI_API_KEY"] = ""

##### UDFs

In [5]:
# docx converters
def docx_to_plain_with_simple_tables(path: str) -> str:
    return pypandoc.convert_file(path, to="plain+simple_tables", format="docx")

def pptx_to_markdown(pptx_file):
    prs = Presentation(pptx_file)
    md_content = []

    for i, slide in enumerate(prs.slides, 1):
        md_content.append(f"## Slide {i}\n")

        for shape in slide.shapes:
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    if paragraph.text.strip():
                        md_content.append(paragraph.text + "\n")

            elif shape.has_table:
                table = shape.table
                table_data = []
                for row in table.rows:
                    row_data = [cell.text.strip() for cell in row.cells]
                    table_data.append(row_data)

                if table_data:
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])
                    md_content.append(df.to_markdown(index=False) + "\n\n")

        md_content.append("---\n\n")

    return ''.join(md_content)
# docx converters

# field extraction
def extract_field(text, prompt, counter=0, model="o3") -> dict:

    client = OpenAI()

    system_prompt = ("Ты эксперт по технологическим стартапам и смотришь презентацию стартапа для Банка")

    schema = {"name": "startup_analysis", "schema": {"type": "object", "properties": {"company_name": {"type": "string"}, "tech_product": {"type": "string"},\
                "tasks": {"type": "string"}, "company_projects": {"type": "string"}, "target_audience": {"type": "string"},
                "target_audience_companies": {"type": "string"}, "target_audience_people": {"type": "string"}, "bank_profit": {"type": "string"}, "metrics": {"type": "string"},
                "achievments": {"type": "string"}},

            "required": ["company_name", "tech_product", "tasks", "company_projects", "target_audience", "target_audience_companies",\
                         "target_audience_people", "bank_profit", "metrics", "achievments"], "additionalProperties": False}, "strict": True}

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {prompt} : "text: {text}"'}]
    response = client.chat.completions.create(model=model, messages=messages, response_format={"type": "json_schema", "json_schema": schema})

    return json.loads(response.choices[0].message.content)

def return_fields(doc_txt):

    result = extract_field(doc_txt, '''извлеки отсюда информацию в JSON формате:
        1. company_name: название компании - если нет, выведи только одно слово 'НЕТ'
        2. tech_product: технологический продукт компании - если нет, выведи только одно слово 'НЕТ'
        3. tasks: Задачи, которые решаются с помощью технологии или продукта без упоминания проекта, продукта и организации - если нет, выведи только одно слово 'НЕТ'
        4. company_projects: реализованные именно компанией проекты (с указанием организации, где был реализован проект) - если нет, выведи только одно слово 'НЕТ'
        5. target_audience: целевая аудитория продукта среди подразделений Банка - если нет, выведи только одно слово 'НЕТ'
        6. target_audience_companies: целевая аудитория продукта среди юридических лиц - если нет, выведи только одно слово 'НЕТ'
        7. target_audience_people: целевая аудитория продукта среди физических лиц - если нет, выведи только одно слово 'НЕТ'
        8. bank_profit: предложение, преимущества или выгода для Банка - если нет, выведи только одно слово 'НЕТ'
        9. metrics: численные бизнес метрики, которые будут достигнуты именно для клиента, а не самой организацией (с цифрами, если они есть в виде чисел, а не слов)\
      - если нет, выведи только одно слово 'НЕТ'
        10. achievments: достижения и награды проекта или продукта (не человека), полученные от других организаций - если нет, выведи только одно слово 'НЕТ''')

    return result
#/field extraction

# parallel computations
def parallel_extraction(texts, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: return_fields(x), texts), total=len(texts)))
    return results
#/parallel computations

##### Load decks

In [6]:
drive.mount('/content/drive')
!cp -r '/content/drive/My Drive/Pitchdecks' '/content/'

Mounted at /content/drive


##### Extract fields

In [7]:
%%time

folder_path = '/content/Pitchdecks'
df_fields, file_texts, adresses = [], [], []

### Extract .docx files
docx_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.docx')])

for docx_file in tqdm(docx_files):

    pptx_path = os.path.join(folder_path, docx_file)
    adresses.append(pptx_path.replace('/content/Питчи/', ''))

    file_texts.append(docx_to_plain_with_simple_tables(pptx_path))
###/Extract .docx files

### Extract .pptx files
pptx_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pptx')])

for pptx_file in tqdm(pptx_files):

    pptx_path = os.path.join(folder_path, pptx_file)
    adresses.append(pptx_path.replace('/content/Питчи/', ''))

    file_texts.append(pptx_to_markdown(pptx_path))
###/Extract .pptx files

### Extract .pdf files
pdf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pdf')])

for pdf_file in tqdm(pdf_files):

    pdf_path = os.path.join(folder_path, pdf_file)
    adresses.append(pdf_path.replace('/content/Питчи/', ''))

    ### Image recognition ###
    pdf_images = convert_from_path(pdf_path, thread_count=10)

    for idx, image in enumerate(pdf_images):
      image.save(f'pdf_page_{idx + 1}.png', 'PNG')

    image_dir = '/content/'
    reader = easyocr.Reader(['ru', 'en'], gpu = True)

    text_results = []

    for filename in sorted(os.listdir(image_dir)):

        if filename.lower().endswith('.png'):

          try:
            img_path = os.path.join(image_dir, filename)
            result = reader.readtext(img_path, detail=0, paragraph=True)
            text_results.extend(result)
          except:
            print("Can't recognize the image")

    img_text = '\n'.join(text_results)
    !rm /content/*.png
    ###/Image recognition ###

    # Extract text with PDF reader
    with pdfplumber.open(pdf_path) as pdf:
      text = ''
      for page in pdf.pages:
          text += page.extract_text() + '\n'

    doc_txt = text.replace('02.12.2024, 22:35 Документ от auto-coder.net', '')
    #/Extract text with PDF reader

    file_texts.append(img_text + '|Краткое содержание текста: ' + doc_txt)

df_fields = list(parallel_extraction(file_texts))

values = [[d.get(k, 'НЕТ') for k in df_fields[0].keys()] for d in df_fields]
df_fields = pd.DataFrame(values, columns=df_fields[0].keys())

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/42 [00:00<?, ?it/s]WARNING:easyocr.easyocr:Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

100%|██████████| 42/42 [01:57<00:00,  2.80s/it]


CPU times: user 24min 3s, sys: 32.1 s, total: 24min 35s
Wall time: 29min 52s


##### Create dataframe:

In [14]:
df = df_fields.copy()

columns=['Название компании', 'Продукт', 'Решаемые продуктом задачи', 'Реализованные проекты', 'Целевая аудитория - Банк', 'Целевая аудитория - юридические лица',\
         'Целевая аудитория - физические лица', 'Оффер для Банка', 'Метрики', 'Достижения']

df.columns = columns

df['Файл'] = adresses
df['Файл'] = df['Файл'].str.split('/').str[-1]

cols = ['Файл'] + [col for col in df.columns if col != 'Файл']
df = df[cols]

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: x[0].upper() + x[1:] if isinstance(x, str) and x and not x[0].isupper() else x)

print(df.to_markdown(index=False, tablefmt="grid"))

+-----------------------------------------------------------------------------+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Save to xlsx:

In [15]:
df.to_excel('pitches - ЦРИП.xlsx', index=False)
files.download('pitches - ЦРИП.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>